## The Machine Learning Part with Monthly Data

First import any lib, we will install Pandasql, statsmodels, and fbprophet via PyPl on Databricks.  


Python Lib Ref:  
- Pandasql: https://pypi.org/project/pandasql/  
- Statsmodel: https://www.statsmodels.org/stable/index.html


Below are some references that might be useful for doing time series analysis:  
 
1) https://otexts.com/fpp2/regression.html  
2) https://www.machinelearningplus.com/time-series/time-series-analysis-python/  
3) https://www.machinelearningplus.com/time-series/arima-model-time-series-forecasting-python/  
4) https://machinelearningmastery.com/decompose-time-series-data-trend-seasonality/  
5) https://facebook.github.io/prophet/docs/quick_start.html#python-api

In [3]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#pip install -U pandasql
from pandasql import sqldf

#pip install -U statsmodels
from statsmodels.tsa.seasonal import seasonal_decompose

#pip install fbprophet
from fbprophet import Prophet


import logging
logger = spark._jvm.org.apache.log4j
logging.getLogger("py4j").setLevel(logging.ERROR)

INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.
INFO:py4j.java_gateway:Received command c on object id p0

Now we load the data we previously prepared

In [5]:
df_MSC_MonthSalesTemp = pd.read_csv("/dbfs/FileStore/tables/MSC_MonthSalesTemp.csv")

df_MSC_MonthSalesTemp['Month'] =  pd.to_datetime(df_MSC_MonthSalesTemp['Month'], format='%Y-%m')

df_MSC_MonthSalesTemp.index = pd.to_datetime(df_MSC_MonthSalesTemp.Month)

print(df_MSC_MonthSalesTemp.shape)
print(df_MSC_MonthSalesTemp.head())
print(df_MSC_MonthSalesTemp.dtypes)


(34, 4)
 Month MonthKRub MonItemSold MonAvgTemp
Month 
2013-01-01 2013-01-01 32739.59204 47495.0 -9.156667
2013-02-01 2013-02-01 33098.88780 46744.0 -4.084735
2013-03-01 2013-03-01 38682.50245 54830.0 -7.410921
2013-04-01 2013-04-01 26048.29295 41348.0 5.498007
2013-05-01 2013-05-01 23351.54330 38878.0 16.610582
Month datetime64[ns]
MonthKRub float64
MonItemSold float64
MonAvgTemp float64
dtype: object

In [6]:
#Check any missing values

df_MSC_MonthSalesTemp.isnull().sum()



Out[3]: Month 0
MonthKRub 0
MonItemSold 0
MonAvgTemp 0
dtype: int64

We should review the data by plotting them against the dates

In [8]:
def plot_df(x, y, title="", xlabel='', ylabel='', dpi=100):
    plt.figure(figsize=(16,5), dpi=dpi)
    plt.plot(x, y, color='tab:red')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    display()

plot_df(x=df_MSC_MonthSalesTemp.Month, y=df_MSC_MonthSalesTemp.MonAvgTemp, xlabel='Month', ylabel='Temperature', title='Monthly Temperature VS Dates') 

plot_df(x=df_MSC_MonthSalesTemp.Month, y=df_MSC_MonthSalesTemp.MonthKRub, xlabel='Month', ylabel='Revenue(kRUB)', title='Monthly Sales Revenue(kRUB) VS Dates')

plot_df(x=df_MSC_MonthSalesTemp.Month, y=df_MSC_MonthSalesTemp.MonItemSold, xlabel='Month', ylabel='Count', title='Monthly Sales Count VS Dates')



From above plots, we can observe that there may be some seasonality and trends. We will decompose latter below.

Next, we will do ADF test. With ADF, the null hypothesis is that a unit root is present in a time series sample. The alternative hypothesis is different depending on which version of the test is used, but is usually stationarity or trend-stationarity. It is an augmented version of the Dickey–Fuller test for a larger and more complicated set of time series models.

In [10]:
from statsmodels.tsa.stattools import adfuller

# ADF Test for monthly temperature
result_adf_tempadd = adfuller(df_MSC_MonthSalesTemp.MonAvgTemp, autolag='AIC')
print(f'ADF Statistic: {result_adf_tempadd[0]}')
print(f'p-value: {result_adf_tempadd[1]}')
for key, value in result_adf_tempadd[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')



/databricks/python/lib/python3.7/site-packages/patsy/constraint.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 from collections import Mapping
/databricks/python/lib/python3.7/site-packages/statsmodels/base/wrapper.py:100: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
 defaults=argspec[3])
ADF Statistic: -5.239224300261936
p-value: 7.307702939994499e-06
Critial Values:
 1%, -3.7112123008648155
Critial Values:
 5%, -2.981246804733728
Critial Values:
 10%, -2.6300945562130176

In [11]:
# ADF Test for monthly revenue
result_adf_mradd = adfuller(df_MSC_MonthSalesTemp.MonthKRub, autolag='AIC')
print(f'ADF Statistic: {result_adf_mradd[0]}')
print(f'p-value: {result_adf_mradd[1]}')
for key, value in result_adf_mradd[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')





ADF Statistic: -4.256355712801631
p-value: 0.0005282031551493868
Critial Values:
 1%, -3.7377092158564813
Critial Values:
 5%, -2.9922162731481485
Critial Values:
 10%, -2.635746736111111

In [12]:


# ADF Test for monthly items count
result_adf_itemsadd = adfuller(df_MSC_MonthSalesTemp.MonItemSold, autolag='AIC')
print(f'ADF Statistic: {result_adf_itemsadd[0]}')
print(f'p-value: {result_adf_itemsadd[1]}')
for key, value in result_adf_itemsadd[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')





ADF Statistic: -2.312079596307234
p-value: 0.16810165367004737
Critial Values:
 1%, -3.6461350877925254
Critial Values:
 5%, -2.954126991123355
Critial Values:
 10%, -2.6159676124885216

From above ADF test, only the monthly items sold (count) series has p-value greater than 0.05, so will be the only one non-stationary, while the monthly average temp and monthly revenue both somewhat stationary.

In this case, we will still do a decompose of all series.

Now we will decompose the raw data.

Additive time series:
Value = Base Level + Trend + Seasonality + Error 

Multiplicative Time Series:
Value = Base Level x Trend x Seasonality x Error

From the Daily ML part, we learnt that both of above will work so let's use the additive one.

In [15]:
# Additive Decomposition for Average Temp
result_mtemp_add = seasonal_decompose(df_MSC_MonthSalesTemp['MonAvgTemp'], model='additive', freq=12)

pd.plotting.register_matplotlib_converters()
result_mtemp_add.plot()
display()



In [16]:
# Additive Decomposition for daily revenue
result_mr_add = seasonal_decompose(df_MSC_MonthSalesTemp['MonthKRub'], model='additive', freq=12)


result_mr_add.plot()
display()



In [17]:
# Additive Decomposition for # of Items Sold
result_items_add = seasonal_decompose(df_MSC_MonthSalesTemp['MonItemSold'], model='additive', freq=12)


result_items_add.plot()
display()



Next, we will reconstruct the data set with different compoenents.

In [19]:
df_recons_mtemp_add = pd.concat([result_mtemp_add.seasonal, result_mtemp_add.trend, result_mtemp_add.resid, result_mtemp_add.observed], axis=1)
df_recons_mtemp_add.columns = ['seas', 'trend', 'resid', 'actual_values']

print(df_recons_mtemp_add.shape)
print(df_recons_mtemp_add.isnull().sum())
print(df_recons_mtemp_add)



(34, 4)
seas 0
trend 12
resid 12
actual_values 0
dtype: int64
 seas trend resid actual_values
Month 
2013-01-01 -13.947908 NaN NaN -9.156667
2013-02-01 -9.155254 NaN NaN -4.084735
2013-03-01 -4.957845 NaN NaN -7.410921
2013-04-01 -0.909834 NaN NaN 5.498007
2013-05-01 8.796962 NaN NaN 16.610582
2013-06-01 9.202085 NaN NaN 19.743669
2013-07-01 13.176186 6.325375 -0.698967 18.802595
2013-08-01 11.980430 6.385677 -0.069631 18.296476
2013-09-01 4.265476 6.857434 -1.013188 10.109722
2013-10-01 -2.301066 7.284489 1.258515 6.241938
2013-11-01 -6.004560 7.270070 2.399768 3.665278
2013-12-01 -10.144671 7.053893 0.852875 -2.237903
2014-01-01 -13.947908 6.949759 -2.505591 -9.503740
2014-02-01 -9.155254 7.037136 -0.172305 -2.290423
2014-03-01 -4.957845 7.133831 -0.059041 2.116945
2014-04-01 -0.909834 7.077918 0.051373 6.219457
2014-05-01 8.796962 6.713491 0.032621 15.543074
2014-06-01 9.202085 6.388224 0.032621 15.622930
2014-07-01 13.176186 6.483728 0.764209 20.424123
2014-08-01 11.980430 6.656678 0.134873 18.771981
2014-09-01 4.265476 6.610987 1.078430 11.954893
2014-10-01 -2.301066 6.549206 -1.193273 3.054868
2014-11-01 -6.004560 6.445193 -2.334526 -1.893893
2014-12-01 -10.144671 6.447160 -0.787633 -4.485144
2015-01-01 -13.947908 6.412663 2.570833 -4.964412
2015-02-01 -9.155254 6.238760 0.237547 -2.678947
2015-03-01 -4.957845 6.242433 0.124283 1.408871
2015-04-01 -0.909834 6.340770 0.013869 5.444805
2015-05-01 8.796962 NaN NaN 13.821401
2015-06-01 9.202085 NaN NaN 17.391813
2015-07-01 13.176186 NaN NaN 17.827327
2015-08-01 11.980430 NaN NaN 17.195104
2015-09-01 4.265476 NaN NaN 13.619902
2015-10-01 -2.301066 NaN NaN 3.749953

In [20]:
df_recons_mr_add = pd.concat([result_mr_add.seasonal, result_mr_add.trend, result_mr_add.resid, result_mr_add.observed], axis=1)
df_recons_mr_add.columns = ['seas', 'trend', 'resid', 'actual_values']

print(df_recons_mr_add.shape)
print(df_recons_mr_add.isnull().sum())
print(df_recons_mr_add)



(34, 4)
seas 0
trend 12
resid 12
actual_values 0
dtype: int64
 seas trend resid actual_values
Month 
2013-01-01 -125.982905 NaN NaN 32739.59204
2013-02-01 -4613.211564 NaN NaN 33098.88780
2013-03-01 -3327.324106 NaN NaN 38682.50245
2013-04-01 -5790.187957 NaN NaN 26048.29295
2013-05-01 -6193.577868 NaN NaN 23351.54330
2013-06-01 -9466.704737 NaN NaN 25718.10220
2013-07-01 -12812.177650 34880.564465 1414.797775 23483.18459
2013-08-01 -9556.877554 34858.766611 -53.457136 25248.43192
2013-09-01 3473.051721 34730.167848 3813.137931 42016.35750
2013-10-01 191.692059 34638.407956 -69.754615 34760.34540
2013-11-01 10847.754693 34978.903938 -497.171631 45329.48700
2013-12-01 37373.545868 35312.804596 -3960.920464 68725.43000
2014-01-01 -125.982905 35381.073642 -3786.265837 31468.82490
2014-02-01 -4613.211564 35525.616020 2934.101984 33846.50644
2014-03-01 -3327.324106 35466.187318 2709.650288 34848.51350
2014-04-01 -5790.187957 35395.503347 -1925.270889 27680.04450
2014-05-01 -6193.577868 35675.136053 410.137116 29891.69530
2014-06-01 -9466.704737 36248.133622 410.137116 27191.56600
2014-07-01 -12812.177650 37054.879093 -594.523544 23648.17790
2014-08-01 -9556.877554 37235.601857 873.731367 28552.45567
2014-09-01 3473.051721 36805.856878 -2992.863700 37286.04490
2014-10-01 191.692059 36712.521795 890.028846 37794.24270
2014-11-01 10847.754693 36841.574095 1317.445862 49006.77465
2014-12-01 37373.545868 36645.343437 4781.194695 78800.08400
2015-01-01 -125.982905 36275.505057 4606.540068 40756.06222
2015-02-01 -4613.211564 35623.654767 -2113.827753 28896.61545
2015-03-01 -3327.324106 34701.225163 -1889.376057 29484.52500
2015-04-01 -5790.187957 33848.633837 2745.545121 30803.99100
2015-05-01 -6193.577868 NaN NaN 29865.00400
2015-06-01 -9466.704737 NaN NaN 22508.72150
2015-07-01 -12812.177650 NaN NaN 19454.90129
2015-08-01 -9556.877554 NaN NaN 17101.32532
2015-09-01 3473.051721 NaN NaN 26598.86476
2015-10-01 191.692059 NaN NaN 28019.23100

In [21]:
df_recons_items_add = pd.concat([result_items_add.seasonal, result_items_add.trend, result_items_add.resid, result_items_add.observed], axis=1)
df_recons_items_add.columns = ['seas', 'trend', 'resid', 'actual_values']

print(df_recons_items_add.shape)
print(df_recons_items_add.isnull().sum())
print(df_recons_items_add)



(34, 4)
seas 0
trend 12
resid 12
actual_values 0
dtype: int64
 seas trend resid actual_values
Month 
2013-01-01 1565.152778 NaN NaN 47495.0
2013-02-01 -3503.888889 NaN NaN 46744.0
2013-03-01 -2055.138889 NaN NaN 54830.0
2013-04-01 -3547.180556 NaN NaN 41348.0
2013-05-01 -5476.534722 NaN NaN 38878.0
2013-06-01 -4865.993056 NaN NaN 42386.0
2013-07-01 -7112.222222 46439.958333 1459.263889 40787.0
2013-08-01 -2707.388889 45844.375000 681.013889 43818.0
2013-09-01 672.277778 44878.625000 3775.097222 49326.0
2013-10-01 226.611111 43978.625000 -676.236111 43529.0
2013-11-01 3424.673611 43482.833333 -836.506944 46071.0
2013-12-01 23379.631944 42899.708333 -1151.340278 65128.0
2014-01-01 1565.152778 42083.083333 -2274.236111 41374.0
2014-02-01 -3503.888889 41241.041667 833.847222 38571.0
2014-03-01 -2055.138889 40211.500000 1668.638889 39825.0
2014-04-01 -3547.180556 39305.000000 -1004.819444 34753.0
2014-05-01 -5476.534722 38786.000000 264.534722 33574.0
2014-06-01 -4865.993056 38296.458333 264.534722 33695.0
2014-07-01 -7112.222222 37921.416667 -930.194444 29879.0
2014-08-01 -2707.388889 37376.333333 -151.944444 34517.0
2014-09-01 672.277778 36491.750000 -3246.027778 33918.0
2014-10-01 226.611111 35749.083333 1205.305556 37181.0
2014-11-01 3424.673611 35172.750000 1365.576389 39963.0
2014-12-01 23379.631944 34426.958333 1680.409722 59487.0
2015-01-01 1565.152778 33645.541667 2803.305556 38014.0
2015-02-01 -3503.888889 32657.666667 -304.777778 28849.0
2015-03-01 -2055.138889 31511.708333 -1139.569444 28317.0
2015-04-01 -3547.180556 30450.291667 1533.888889 28437.0
2015-05-01 -5476.534722 NaN NaN 26058.0
2015-06-01 -4865.993056 NaN NaN 23312.0
2015-07-01 -7112.222222 NaN NaN 21508.0
2015-08-01 -2707.388889 NaN NaN 19179.0
2015-09-01 672.277778 NaN NaN 21753.0
2015-10-01 226.611111 NaN NaN 23872.0

For doing the linear regression, we will find the Co-variance of the monthly temp against monthly sales revenue and items count, with Person Correcltion.  

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html

The Pearson correlation coefficient (named for Karl Pearson) can be used to summarize the strength of the linear relationship between two data samples.

The Pearson’s correlation coefficient is calculated as the covariance of the two variables divided by the product of the standard deviation of each data sample. It is the normalization of the covariance between the two variables to give an interpretable score.

"Correlations of -1 or +1 imply an exact linear relationship. "

In [23]:
#The Pearson’s correlation coefficient for raw data
from scipy.stats import pearsonr

pcovariance_01 = pearsonr(df_MSC_MonthSalesTemp.MonAvgTemp, df_MSC_MonthSalesTemp.MonthKRub)

pcovariance_02 = pearsonr(df_MSC_MonthSalesTemp.MonAvgTemp, df_MSC_MonthSalesTemp.MonItemSold)

print(pcovariance_01)
print(pcovariance_02)



(-0.5559021948565455, 0.0006415851692779342)
(-0.49027539700215794, 0.003244987344119639)

We can do a plot of Avg Monthly Temp VS Monthly Revenue, for both raw and decomposed data

In [25]:
def plot_pts(x, y, title="", xlabel='', ylabel='', dpi=100):
    plt.figure(figsize=(16,5), dpi=dpi)
    plt.plot(x, y, 'bo')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    display()

plot_pts(x=df_MSC_MonthSalesTemp.MonAvgTemp, y=df_MSC_MonthSalesTemp.MonthKRub, 
         title='Monthly Temp VS Monthly Revenue (Raw Data)', xlabel='Average Monthly Temperature', ylabel='Monthly Revenue') 

plot_pts(x=df_recons_mtemp_add.seas, y=df_recons_mr_add.seas, 
         title='Monthly Temp VS Monthly Revenue (Seasonal Decomposed)', xlabel='Average Monthly Temperature', ylabel='Monthly Revenue') 



From above plots, we can see that we might be able to find a linear relationship between the pair of average monthly temperature and revenue. Next we will do some tests for different components with Pearson's correlation coefficient.

In [27]:
#The Pearson’s correlation coefficient for residuals of the reconstructed data

pcovariance_res01 = pearsonr(df_recons_mtemp_add.resid[6:28], df_recons_mr_add.resid[6:28])

pcovariance_res02 = pearsonr(df_recons_mtemp_add.resid[6:28], df_recons_items_add.resid[6:28])

print(pcovariance_res01)
print(pcovariance_res02)




(-0.02097412308775772, 0.9261858681329244)
(-0.15690561661171779, 0.4855951857173506)

In [28]:
#The Pearson’s correlation coefficient for seasonality of the reconstructed data

pcovariance_res01 = pearsonr(df_recons_mtemp_add.seas[6:28], df_recons_mr_add.seas[6:28])

pcovariance_res02 = pearsonr(df_recons_mtemp_add.seas[6:28], df_recons_items_add.seas[6:28])

print(pcovariance_res01)
print(pcovariance_res02)



(-0.5569134105251609, 0.007097392554202202)
(-0.5269722935420204, 0.01173740087874103)

In [29]:
#The Pearson’s correlation coefficient for trends of the reconstructed data

pcovariance_res01 = pearsonr(df_recons_mtemp_add.trend[6:28], df_recons_mr_add.trend[6:28])

pcovariance_res02 = pearsonr(df_recons_mtemp_add.trend[6:28], df_recons_items_add.trend[6:28])

print(pcovariance_res01)
print(pcovariance_res02)




(-0.21416545069984325, 0.3385387801180597)
(0.5634838226510864, 0.006316342252118516)

From the above Pearson correlation coefficients, we can see that they are better than the daily ones. Moreover, the correlation coefficients for raw data is similar to the decomposed residuals. 

We will fit the raw data of monthly avg temp and monthly sales revenue with SKLearn.

In [31]:
#Import relevant libraries

from sklearn.linear_model import LinearRegression
import sklearn.preprocessing


In [32]:


#Linear regression for raw data, between monthly temp and monthly sales revenue

# create X and y
feature_cols_01 = ['MonAvgTemp']

#print(data[feature_cols_01])

X_01 = df_MSC_MonthSalesTemp[feature_cols_01]
y_01 = df_MSC_MonthSalesTemp[['MonthKRub']]


#Check if the features and labels sets are good
print(X_01)
print(y_01)

lm_01 = LinearRegression()
lm_01.fit(X_01, y_01)

lm_01_intercept = lm_01.intercept_
# print intercept and coefficients
print ('Intercept for Monthly Reve LM is:', lm_01_intercept)

lm_01_coef = lm_01.coef_
print ('Coeffcients for Monthly Reve LM is:', lm_01.coef_)



Rsq_01 = lm_01.score(X_01, y_01)
print('R-square for Monthly Reve LM is:', Rsq_01)





MonAvgTemp
Month 
2013-01-01 -9.156667
2013-02-01 -4.084735
2013-03-01 -7.410921
2013-04-01 5.498007
2013-05-01 16.610582
2013-06-01 19.743669
2013-07-01 18.802595
2013-08-01 18.296476
2013-09-01 10.109722
2013-10-01 6.241938
2013-11-01 3.665278
2013-12-01 -2.237903
2014-01-01 -9.503740
2014-02-01 -2.290423
2014-03-01 2.116945
2014-04-01 6.219457
2014-05-01 15.543074
2014-06-01 15.622930
2014-07-01 20.424123
2014-08-01 18.771981
2014-09-01 11.954893
2014-10-01 3.054868
2014-11-01 -1.893893
2014-12-01 -4.485144
2015-01-01 -4.964412
2015-02-01 -2.678947
2015-03-01 1.408871
2015-04-01 5.444805
2015-05-01 13.821401
2015-06-01 17.391813
2015-07-01 17.827327
2015-08-01 17.195104
2015-09-01 13.619902
2015-10-01 3.749953
 MonthKRub
Month 
2013-01-01 32739.59204
2013-02-01 33098.88780
2013-03-01 38682.50245
2013-04-01 26048.29295
2013-05-01 23351.54330
2013-06-01 25718.10220
2013-07-01 23483.18459
2013-08-01 25248.43192
2013-09-01 42016.35750
2013-10-01 34760.34540
2013-11-01 45329.48700
2013-12-01 68725.43000
2014-01-01 31468.82490
2014-02-01 33846.50644
2014-03-01 34848.51350
2014-04-01 27680.04450
2014-05-01 29891.69530
2014-06-01 27191.56600
2014-07-01 23648.17790
2014-08-01 28552.45567
2014-09-01 37286.04490
2014-10-01 37794.24270
2014-11-01 49006.77465
2014-12-01 78800.08400
2015-01-01 40756.06222
2015-02-01 28896.61545
2015-03-01 29484.52500
2015-04-01 30803.99100
2015-05-01 29865.00400
2015-06-01 22508.72150
2015-07-01 19454.90129
2015-08-01 17101.32532
2015-09-01 26598.86476
2015-10-01 28019.23100
Intercept for Monthly Reve LM is: [38334.12930474]
Coeffcients for Monthly Reve LM is: [[-727.95652026]]
R-square for Monthly Reve LM is: 0.3090272502463248

In [33]:
# Do a plot with line of best fit

X_new = [[df_MSC_MonthSalesTemp.MonAvgTemp.min()], [df_MSC_MonthSalesTemp.MonAvgTemp.max()]]
pred = lm_01.predict(X_new)

plt.figure(figsize=(16,5), dpi=100)
plt.plot(df_MSC_MonthSalesTemp.MonAvgTemp, df_MSC_MonthSalesTemp.MonthKRub, 'bo')
plt.gca().set(title='Monthly Temp VS Monthly Revenue', xlabel='Average Monthly Temperature', ylabel='Monthly Revenue (kRUB)')

plt.plot(X_new, pred, c='red', linewidth=1)

display()



From above plot with line of best fit, there may be some outliers, so we need to do something to check them. We can use [seaborn package](https://seaborn.pydata.org/), for example as below.

In [35]:
import seaborn as sns
sns.boxplot(x=df_MSC_MonthSalesTemp['MonthKRub'])

display()


From above box plot of the monthly revenue, there are outliers above the 60,000 KRub, those are the sales data points from the two Decembers.

In [37]:
#Linear regression for raw data, between monthly temp and monthly sales revenue, after removing outliers

# create X and y

X_02 = X_01.copy()
y_02 = y_01.copy()

X_02 = X_02.drop([X_02.index[11], X_02.index[23]])
y_02 = y_02.drop([y_02.index[11], y_02.index[23]])

#Check if the features and labels sets are good
print(X_02)
print(y_02)

lm_02 = LinearRegression()
lm_02.fit(X_02, y_02)

lm_02_intercept = lm_02.intercept_
# print intercept and coefficients
print ('Intercept for Monthly Reve LM is:', lm_02_intercept)

lm_02_coef = lm_02.coef_
print ('Coeffcients for Monthly Reve LM is:', lm_02.coef_)



Rsq_02 = lm_02.score(X_02, y_02)
print('R-square for Monthly Reve LM is:', Rsq_02)



MonAvgTemp
Month 
2013-01-01 -9.156667
2013-02-01 -4.084735
2013-03-01 -7.410921
2013-04-01 5.498007
2013-05-01 16.610582
2013-06-01 19.743669
2013-07-01 18.802595
2013-08-01 18.296476
2013-09-01 10.109722
2013-10-01 6.241938
2013-11-01 3.665278
2014-01-01 -9.503740
2014-02-01 -2.290423
2014-03-01 2.116945
2014-04-01 6.219457
2014-05-01 15.543074
2014-06-01 15.622930
2014-07-01 20.424123
2014-08-01 18.771981
2014-09-01 11.954893
2014-10-01 3.054868
2014-11-01 -1.893893
2015-01-01 -4.964412
2015-02-01 -2.678947
2015-03-01 1.408871
2015-04-01 5.444805
2015-05-01 13.821401
2015-06-01 17.391813
2015-07-01 17.827327
2015-08-01 17.195104
2015-09-01 13.619902
2015-10-01 3.749953
 MonthKRub
Month 
2013-01-01 32739.59204
2013-02-01 33098.88780
2013-03-01 38682.50245
2013-04-01 26048.29295
2013-05-01 23351.54330
2013-06-01 25718.10220
2013-07-01 23483.18459
2013-08-01 25248.43192
2013-09-01 42016.35750
2013-10-01 34760.34540
2013-11-01 45329.48700
2014-01-01 31468.82490
2014-02-01 33846.50644
2014-03-01 34848.51350
2014-04-01 27680.04450
2014-05-01 29891.69530
2014-06-01 27191.56600
2014-07-01 23648.17790
2014-08-01 28552.45567
2014-09-01 37286.04490
2014-10-01 37794.24270
2014-11-01 49006.77465
2015-01-01 40756.06222
2015-02-01 28896.61545
2015-03-01 29484.52500
2015-04-01 30803.99100
2015-05-01 29865.00400
2015-06-01 22508.72150
2015-07-01 19454.90129
2015-08-01 17101.32532
2015-09-01 26598.86476
2015-10-01 28019.23100
Intercept for Monthly Reve LM is: [34310.29300885]
Coeffcients for Monthly Reve LM is: [[-467.54151937]]
R-square for Monthly Reve LM is: 0.3691259045697623

In [38]:
# Do a new plot with new line of best fit

X_new = [[df_MSC_MonthSalesTemp.MonAvgTemp.min()], [df_MSC_MonthSalesTemp.MonAvgTemp.max()]]
pred = lm_02.predict(X_new)

plt.figure(figsize=(16,5), dpi=100)
plt.plot(X_02, y_02, 'bo')
plt.gca().set(title='Monthly Temp VS Monthly Revenue', xlabel='Average Monthly Temperature', ylabel='Monthly Revenue (kRUB)')

plt.plot(X_new, pred, c='red', linewidth=1)

display()

From above new fit and new plot, the R-square is still not so great, looks like we need to do some scaling. Let's try taking the log for the monthly revenue.

In [40]:
#Linear regression for raw data, between monthly temp and monthly sales revenue, with scaling

# create X and y

X_03 = X_02.copy()
y_03 = y_02.copy()

y_03['MonthKRub'] = np.log(y_03['MonthKRub'])

#Check if the features and labels sets are good
print(X_03)
print(y_03)

lm_03 = LinearRegression()
lm_03.fit(X_03, y_03)

lm_03_intercept = lm_03.intercept_
# print intercept and coefficients
print ('Intercept for Monthly Reve LM is:', lm_03_intercept)

lm_03_coef = lm_03.coef_
print ('Coeffcients for Monthly Reve LM is:', lm_03.coef_)



Rsq_03 = lm_03.score(X_03, y_03)
print('R-square for Monthly Reve LM is:', Rsq_03)



MonAvgTemp
Month 
2013-01-01 -9.156667
2013-02-01 -4.084735
2013-03-01 -7.410921
2013-04-01 5.498007
2013-05-01 16.610582
2013-06-01 19.743669
2013-07-01 18.802595
2013-08-01 18.296476
2013-09-01 10.109722
2013-10-01 6.241938
2013-11-01 3.665278
2014-01-01 -9.503740
2014-02-01 -2.290423
2014-03-01 2.116945
2014-04-01 6.219457
2014-05-01 15.543074
2014-06-01 15.622930
2014-07-01 20.424123
2014-08-01 18.771981
2014-09-01 11.954893
2014-10-01 3.054868
2014-11-01 -1.893893
2015-01-01 -4.964412
2015-02-01 -2.678947
2015-03-01 1.408871
2015-04-01 5.444805
2015-05-01 13.821401
2015-06-01 17.391813
2015-07-01 17.827327
2015-08-01 17.195104
2015-09-01 13.619902
2015-10-01 3.749953
 MonthKRub
Month 
2013-01-01 10.396340
2013-02-01 10.407255
2013-03-01 10.563143
2013-04-01 10.167708
2013-05-01 10.058418
2013-06-01 10.154950
2013-07-01 10.064040
2013-08-01 10.136519
2013-09-01 10.645814
2013-10-01 10.456233
2013-11-01 10.721713
2014-01-01 10.356753
2014-02-01 10.429591
2014-03-01 10.458766
2014-04-01 10.228467
2014-05-01 10.305336
2014-06-01 10.210662
2014-07-01 10.071041
2014-08-01 10.259498
2014-09-01 10.526374
2014-10-01 10.539912
2014-11-01 10.799714
2015-01-01 10.615360
2015-02-01 10.271480
2015-03-01 10.291621
2015-04-01 10.335400
2015-05-01 10.304443
2015-06-01 10.021658
2015-07-01 9.875854
2015-08-01 9.746911
2015-09-01 10.188624
2015-10-01 10.240646
Intercept for Monthly Reve LM is: [10.42711215]
Coeffcients for Monthly Reve LM is: [[-0.01582962]]
R-square for Monthly Reve LM is: 0.4025378905351942

In [41]:
# Do a new plot with new line of best fit for log(kRub)

X_new = [[df_MSC_MonthSalesTemp.MonAvgTemp.min()], [df_MSC_MonthSalesTemp.MonAvgTemp.max()]]
pred = lm_03.predict(X_new)

plt.figure(figsize=(16,5), dpi=100)
plt.plot(X_03, y_03, 'bo')
plt.gca().set(title='Monthly Temp VS Log of Monthly Revenue', xlabel='Average Monthly Temperature', ylabel='Monthly Revenue log(kRUB)')

plt.plot(X_new, pred, c='red', linewidth=1)

display()

So above fitting and plots for line-of-best-fit with log-transform show some improvements, with new R-square value to be 0.403.

Next, we will proceed with time series forecast with the Facebook Prophet 

https://facebook.github.io/prophet/docs/quick_start.html#python-api

Note we use first 30 months of data [0:30] as the training set, and [30:34] as the validation/test set

In [44]:
df_fb_montemp = pd.concat([df_MSC_MonthSalesTemp.Month[0:30], df_MSC_MonthSalesTemp.MonAvgTemp[0:30]], axis=1)
df_fb_montemp.columns = ['ds', 'y']

df_val_montemp = pd.concat([df_MSC_MonthSalesTemp.Month[30:34], df_MSC_MonthSalesTemp.MonAvgTemp[30:34]], axis=1)
df_val_montemp.columns = ['ds', 'y_act']

true_montemp = df_val_montemp['y_act']

print(true_montemp)
#print(df_fb_montemp.head())
#print(df_fb_montemp.shape)

#print(df_fb_montemp.ds)

m_temp = Prophet()
#m = Prophet(holidays=holidays)
#m.add_country_holidays(country_name='RU')


m_temp.fit(df_fb_montemp)



Month
2015-07-01 17.827327
2015-08-01 17.195104
2015-09-01 13.619902
2015-10-01 3.749953
Name: y_act, dtype: float64
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 23.
Out[27]: <fbprophet.forecaster.Prophet at 0x7f463ada8048>

In [45]:
future_temp = m_temp.make_future_dataframe(periods = 18, freq = 'MS')  

forecast_temp = m_temp.predict(future_temp)
m_temp.plot(forecast_temp)

display()

In [46]:
# Check the y-hats for monthly temp that will be used to calculate RMSE

print(forecast_temp[['ds','yhat']][30:34])

pred_temp = forecast_temp['yhat'][30:34]
print(pred_temp)


ds yhat
30 2015-07-01 22.175109
31 2015-08-01 21.223022
32 2015-09-01 17.227286
33 2015-10-01 3.932157
30 22.175109
31 21.223022
32 17.227286
33 3.932157
Name: yhat, dtype: float64

In [47]:
m_temp.plot_components(forecast_temp)

display()

Above plot looks more promising than the daily ones, let's see the stationarity

In [49]:
df_fb_monre = pd.concat([df_MSC_MonthSalesTemp.Month[0:30], df_MSC_MonthSalesTemp.MonthKRub[0:30]], axis=1)
df_fb_monre.columns = ['ds', 'y']

df_val_monre = pd.concat([df_MSC_MonthSalesTemp.Month[30:34], df_MSC_MonthSalesTemp.MonthKRub[30:34]], axis=1)
df_val_monre.columns = ['ds', 'y_act']
true_monre = df_val_monre['y_act']

print(true_monre)

m_monre = Prophet()
#m = Prophet(holidays=holidays)
#m.add_country_holidays(country_name='RU')


m_monre.fit(df_fb_monre)



Month
2015-07-01 19454.90129
2015-08-01 17101.32532
2015-09-01 26598.86476
2015-10-01 28019.23100
Name: y_act, dtype: float64
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 23.
Out[31]: <fbprophet.forecaster.Prophet at 0x7f463dfab6a0>

In [50]:
future_monre = m_monre.make_future_dataframe(periods = 18, freq = 'MS')  

forecast_monre = m_monre.predict(future_temp)
m_monre.plot(forecast_monre)

display()



In [51]:
m_monre.plot_components(forecast_monre)

display()



In [52]:
# Check the y-hats for monthly revenue that will be used to calculate RMSE

print(forecast_monre[['ds','yhat']][30:34])

pred_monre = forecast_monre['yhat'][30:34]
print(pred_monre)



ds yhat
30 2015-07-01 22216.871467
31 2015-08-01 28156.840699
32 2015-09-01 26567.912125
33 2015-10-01 32476.449794
30 22216.871467
31 28156.840699
32 26567.912125
33 32476.449794
Name: yhat, dtype: float64

In [53]:
from sklearn.metrics import mean_squared_error

mse_temp = mean_squared_error(true_montemp, pred_temp)
rmse_temp = mse_temp**(0.5)

mse_monre = mean_squared_error(true_monre, pred_monre)
rmse_monre = mse_monre**(0.5)

print('The RMSE for Monthly Temperature is:', rmse_temp)

print('The RMSE for Monthly Revenue is:', rmse_monre)

The RMSE for Monthly Temperature is: 3.4703651650487926
The RMSE for Monthly Revenue is: 6118.019634748551

The RMSE for Monthly Temperature is: 3.4703651650487926  
The RMSE for Monthly Revenue is: 6118.019634748551


From Data Preparation Notebook, we know the following means and standard deviations:

Means:  
MonthKRub:      33314.892034  
MonItemSold:    37540.470588  
MonAvgTemp:         6.894968  

Standard Deviations:  
MonthKRub:      12533.036939  
MonItemSold:    10755.099507  
MonAvgTemp:         9.570823  

In this case, the RMSEs for monthly temperature and revenue as predicted by the Facebook Prophet is not that bad, but not great either.